# DECISION TREE Y RANDOM FOREST
---

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('once')

/home/sanfermen/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
df_est = pd.read_csv("../datos/06-bikes_enc_est_limpio.csv", index_col = 0)
df_est.head()

,season,yr,mnth,holiday,weekday,workingday,weathersit,atemp,hum,windspeed,registered
0,0,1,0,0,0,1,2,-0.680818,1.252343,-0.387833,654
1,0,1,0,7,2,2,2,-0.741507,0.480996,0.748899,670
2,0,1,1,7,2,2,4,-1.750344,-1.338073,0.745931,1229
3,1,1,1,7,1,1,4,-1.610886,-0.261577,-0.389769,1454
4,1,1,2,7,1,2,4,-1.505615,-1.340294,-0.046477,1518


In [5]:
df_est.isnull().sum()

season        0
yr            0
mnth          0
holiday       0
weekday       0
workingday    0
weathersit    0
atemp         0
hum           0
windspeed     0
registered    0
dtype: int64

In [12]:
# Creamos una clase para poder hacer el decision tree y el random forest más rápido
class Regres_lineal:
    def __init__(self, dataf, col_pred):
        
        self.dataf = dataf
        self.col_pred = col_pred

    def tree_param(self):
        X = self.dataf.drop(self.col_pred, axis = 1)
        y = self.dataf[self.col_pred]

        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

        arbol = DecisionTreeRegressor(random_state =0)
    
        arbol.fit(x_train, y_train)

        max_feat = np.sqrt(len(x_train.columns))
        max_dep = arbol.tree_.max_depth
        return(max_feat, max_dep)

    def metricas(self, y_te, y_tr, y_te_pred, y_tr_pred, tipo_modelo):
        resultados = {'MAE': [mean_absolute_error(y_te, y_te_pred), mean_absolute_error(y_tr, y_tr_pred)],
                    'MSE': [mean_squared_error(y_te, y_te_pred), mean_squared_error(y_tr, y_tr_pred)],
                    'RMSE': [np.sqrt(mean_squared_error(y_te, y_te_pred)), np.sqrt(mean_squared_error(y_tr, y_tr_pred))],
                    'R2':  [r2_score(y_te, y_te_pred), r2_score(y_tr, y_tr_pred)],
                    "set": ["test", "train"]}
        df = pd.DataFrame(resultados)
        df["modelo"] = tipo_modelo
        return df

    def grid_s_gradient(self, dict, model):
        self.dict = dict
        self.model = model
        
        X = self.dataf.drop(self.col_pred, axis = 1)
        y = self.dataf[self.col_pred]
        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
        
        gs = GridSearchCV(
            estimator= GradientBoostingRegressor(), 
            param_grid= self.dict, 
            cv=10, 
            verbose=-1,
            return_train_score = True,
            scoring="neg_mean_squared_error")
        gs.fit(x_train, y_train)
        mejor_modelo = gs.best_estimator_
        y_pred_test_dt2 = mejor_modelo.predict(x_test)
        y_pred_train_dt2 = mejor_modelo.predict(x_train)
        dt_results = self.metricas(y_test, y_train, y_pred_test_dt2, y_pred_train_dt2, self.model)
        return mejor_modelo, dt_results

    def grid_s_forest(self, dict, model):
        self.dict = dict
        self.model = model
        
        X = self.dataf.drop(self.col_pred, axis = 1)
        y = self.dataf[self.col_pred]
        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
        
        gs = GridSearchCV(
            estimator= RandomForestRegressor(), 
            param_grid= self.dict, 
            cv=10, 
            verbose=-1,
            return_train_score = True,
            scoring="neg_mean_squared_error")
        gs.fit(x_train, y_train)
        mejor_modelo = gs.best_estimator_
        y_pred_test_dt2 = mejor_modelo.predict(x_test)
        y_pred_train_dt2 = mejor_modelo.predict(x_train)
        dt_results = self.metricas(y_test, y_train, y_pred_test_dt2, y_pred_train_dt2, self.model)
        return mejor_modelo, dt_results

In [15]:
# Llamamos a la clase
rg_est = Regres_lineal(df_est, "registered")

In [16]:
features_est, depth_est = rg_est.tree_param()

In [17]:
features_est #El número máx será 3

3.1622776601683795

In [18]:
depth_est # La profundidad máxima será 20

20

### Empezamos las diferentes pruebas

In [11]:
param1 = {"max_depth": [16, 18, 20], 
        "max_features": [3,4],
        "min_samples_split": [50, 100],
        "min_samples_leaf": [50,100]} 

In [19]:
best_model1, result1 = rg_est.grid_s_gradient(param1, "GradientBoosting Estand I")

In [20]:
best_model1

GradientBoostingRegressor(max_depth=18, max_features=3, min_samples_leaf=50,
                          min_samples_split=100)

In [21]:
result1

,MAE,MSE,RMSE,R2,set,modelo
0,528.744249,470780.206436,686.134248,0.781902,test,GradientBoosting Estand I
1,494.071654,408057.741140,638.793974,0.836324,train,GradientBoosting Estand I


In [22]:
param2 = {"max_depth": [4,6,8,10], 
        "max_features": [2, 3,4],
        "min_samples_split": [ 40, 50, 60],
        "min_samples_leaf": [10, 20]} 

In [23]:
best_model2, result2 = rg_est.grid_s_gradient(param2, "GradientBoosting Estand II")

In [24]:
best_model2

GradientBoostingRegressor(max_depth=8, max_features=2, min_samples_leaf=10,
                          min_samples_split=50)

In [25]:
result2

,MAE,MSE,RMSE,R2,set,modelo
0,580.728054,543115.262001,736.963542,0.748391,test,GradientBoosting Estand II
1,335.712951,199171.231542,446.286042,0.920110,train,GradientBoosting Estand II


In [26]:
param3 = {"max_depth": [ 12, 14, 16, 18], 
        "max_features": [3,4],
        "min_samples_split": [10, 20, 40],
        "min_samples_leaf": [10, 20, 40]} 

In [27]:
best_model3, result3 = rg_est.grid_s_forest(param3, "RandForest Estand I")

In [28]:
best_model3

RandomForestRegressor(max_depth=16, max_features=4, min_samples_leaf=10,
                      min_samples_split=10)

In [29]:
result3

,MAE,MSE,RMSE,R2,set,modelo
0,591.030333,549304.670722,741.150910,0.745524,test,RandForest Estand I
1,574.582116,521549.202673,722.183635,0.790801,train,RandForest Estand I


In [30]:
param4 = {"max_depth": [12, 14], 
        "max_features": [4,5],
        "min_samples_split": [ 40, 50],
        "min_samples_leaf": [10, 20,]} 

In [31]:
best_model4, result4 = rg_est.grid_s_forest(param4, "RandForest Estand II")

In [32]:
best_model4

RandomForestRegressor(max_depth=12, max_features=5, min_samples_leaf=10,
                      min_samples_split=40)

In [33]:
result4

,MAE,MSE,RMSE,R2,set,modelo
0,623.402922,601058.803343,775.279823,0.721548,test,RandForest Estand II
1,607.792122,586937.208576,766.118273,0.764574,train,RandForest Estand II


### Cargamos el archivo sin estandarizar

In [35]:
df_sinest = pd.read_csv("../datos/05-bikes_enc_sinest_limpio.csv", index_col = 0)
df_sinest.head()

,season,yr,mnth,holiday,weekday,workingday,weathersit,atemp,hum,windspeed,registered
0,0,1,0,0,0,1,2,18.18125,80.5833,10.749882,654
1,0,1,0,7,2,2,2,17.68695,69.6087,16.652113,670
2,0,1,1,7,2,2,4,9.47025,43.7273,16.636703,1229
3,1,1,1,7,1,1,4,10.60610,59.0435,10.739832,1454
4,1,1,2,7,1,2,4,11.46350,43.6957,12.522300,1518


In [36]:
rg_sinest = Regres_lineal(df_sinest, "registered")

In [37]:
features_sinest, depth_sinest = rg_sinest.tree_param()

In [38]:
features_sinest #El número máx será 3

3.1622776601683795

In [39]:
depth_sinest # La profundidad máxima será 20

20

In [41]:
best_model5, result5 = rg_sinest.grid_s_gradient(param1, "GradientBoosting SinEstand I")

In [42]:
result5

,MAE,MSE,RMSE,R2,set,modelo
0,527.223095,480958.611140,693.511796,0.777187,test,GradientBoosting SinEstand I
1,493.949318,404837.229163,636.268205,0.837616,train,GradientBoosting SinEstand I


In [44]:
best_model6, result6 = rg_sinest.grid_s_gradient(param2, "GradientBoosting SinEstand II")

In [45]:
result6

,MAE,MSE,RMSE,R2,set,modelo
0,548.087538,501643.807454,708.268175,0.767604,test,GradientBoosting SinEstand II
1,355.494170,214408.017184,463.042133,0.913999,train,GradientBoosting SinEstand II


In [46]:
best_model7, result7 = rg_sinest.grid_s_forest(param3, "RandForest SinEstand I")

In [47]:
result7

,MAE,MSE,RMSE,R2,set,modelo
0,598.493287,546047.947329,738.950572,0.747033,test,RandForest SinEstand I
1,581.475085,537192.422237,732.934119,0.784527,train,RandForest SinEstand I


In [48]:
best_model8, result8 = rg_sinest.grid_s_forest(param4, "RandForest SinEstand II")

In [49]:
result8

,MAE,MSE,RMSE,R2,set,modelo
0,629.956205,604310.966225,777.374405,0.720041,test,RandForest SinEstand II
1,611.816336,587788.904355,766.673923,0.764232,train,RandForest SinEstand II


In [50]:
param5 = {"max_depth": [22], 
        "max_features": [4],
        "min_samples_split": [8],
        "min_samples_leaf": [8]}

In [51]:
best_model9, result9 = rg_sinest.grid_s_forest(param5, "RandForest SinEstand III")

In [52]:
result9

,MAE,MSE,RMSE,R2,set,modelo
0,587.477563,547185.681343,739.720002,0.746506,test,RandForest SinEstand III
1,550.615589,483816.213449,695.568985,0.805936,train,RandForest SinEstand III


In [53]:
best_model10, result10 = rg_est.grid_s_forest(param5, "RandForest Estand III")

In [54]:
result10

,MAE,MSE,RMSE,R2,set,modelo
0,592.043723,553561.479445,744.017123,0.743552,test,RandForest Estand III
1,549.401541,486437.930957,697.451024,0.804885,train,RandForest Estand III


In [55]:
param6 = {"max_depth": [18,20], 
        "max_features": [3,4],
        "min_samples_split": [8, 15],
        "min_samples_leaf": [8, 15]}

In [56]:
best_model11, result11 = rg_est.grid_s_forest(param6, "RandForest Estand IV")

In [57]:
best_model11

RandomForestRegressor(max_depth=20, max_features=4, min_samples_leaf=8,
                      min_samples_split=8)

In [58]:
result11

,MAE,MSE,RMSE,R2,set,modelo
0,586.774885,546935.069668,739.550586,0.746622,test,RandForest Estand IV
1,547.280409,483955.876153,695.669373,0.805880,train,RandForest Estand IV


In [60]:
best_model12, result12 = rg_est.grid_s_gradient(param5, "GradientBoosting Estand III")

In [61]:
result12

,MAE,MSE,RMSE,R2,set,modelo
0,591.282804,587794.968040,766.677878,0.727693,test,GradientBoosting Estand III
1,102.197311,21050.213171,145.086916,0.991557,train,GradientBoosting Estand III


In [63]:
best_model13, result13 = rg_sinest.grid_s_gradient(param5, "GradientBoosting SinEstand III")

In [64]:
best_model13

GradientBoostingRegressor(max_depth=22, max_features=4, min_samples_leaf=8,
                          min_samples_split=8)

In [65]:
result13

,MAE,MSE,RMSE,R2,set,modelo
0,574.941298,557724.492906,746.809543,0.741623,test,GradientBoosting SinEstand III
1,101.216273,21291.861911,145.917312,0.991460,train,GradientBoosting SinEstand III


In [66]:
param7 = {"max_depth": [18,20], 
        "max_features": [4],
        "min_samples_split": [8, 16],
        "min_samples_leaf": [8]}

In [67]:
best_model14, result14 = rg_est.grid_s_gradient(param7, "GradientBoosting Estand IV")

In [68]:
best_model14

GradientBoostingRegressor(max_depth=18, max_features=4, min_samples_leaf=8,
                          min_samples_split=16)

In [69]:
result14

,MAE,MSE,RMSE,R2,set,modelo
0,593.773667,561112.467350,749.074407,0.740054,test,GradientBoosting Estand IV
1,104.495152,23226.941906,152.403878,0.990683,train,GradientBoosting Estand IV


In [94]:
param17 = {"max_depth": [4], 
        "max_features": [3],
        "min_samples_split": [50],
        "min_samples_leaf": [10]}

best_model15, result15 = rg_est.grid_s_gradient(param17, "GradientBoosting Estand IV")

result15

,MAE,MSE,RMSE,R2,set,modelo
0,560.739106,498220.741229,705.847534,0.769190,test,GradientBoosting Estand IV
1,429.194916,307666.213012,554.676674,0.876592,train,GradientBoosting Estand IV


In [95]:
param17 = {"max_depth": [4], 
        "max_features": [3],
        "min_samples_split": [50],
        "min_samples_leaf": [10]}

best_model16, result16 = rg_sinest.grid_s_gradient(param17, "GradientBoosting SinEstand IV")

result16

,MAE,MSE,RMSE,R2,set,modelo
0,561.969780,500096.241914,707.174831,0.768321,test,GradientBoosting SinEstand IV
1,427.095973,301711.404967,549.282628,0.878981,train,GradientBoosting SinEstand IV
